In [ ]:
# 3_寿命分布建模.ipynb
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import weibull_min, expon, norm, pearsonr
from utils import StatisticalAnalyzer

# 1. 加载实际数据（接第一部分）
lifetimes = results_part1['lifetimes']

# 2. 拟合威布尔分布
weibull_params = StatisticalAnalyzer.fit_weibull_distribution(lifetimes)
k = weibull_params['k']
lam = weibull_params['lambda']

print("=== 威布尔分布拟合结果 ===")
print(f"形状参数 k = {k:.3f}")
print(f"尺度参数 λ = {lam:.1f}")
print(f"平均寿命 E(T) = {lam * np.math.gamma(1 + 1/k):.1f}")
print(f"寿命方差 D(T) = {lam**2 * (np.math.gamma(1 + 2/k) - np.math.gamma(1 + 1/k)**2):.1f}")

# 3. 比较不同分布拟合
plt.figure(figsize=(14, 10))

# 3.1 直方图与分布拟合
plt.subplot(2, 3, 1)
hist_values, bins, _ = plt.hist(lifetimes, bins=20, density=True, alpha=0.6, edgecolor='black')
bin_centers = (bins[:-1] + bins[1:]) / 2

# 威布尔分布
x = np.linspace(0, max(lifetimes), 1000)
pdf_weibull = weibull_min.pdf(x, k, scale=lam)
plt.plot(x, pdf_weibull, 'r-', linewidth=2, label=f'Weibull(k={k:.2f}, λ={lam:.0f})')

# 指数分布（威布尔k=1时的特例）
rate_exp = 1 / np.mean(lifetimes)
pdf_exp = expon.pdf(x, scale=1/rate_exp)
plt.plot(x, pdf_exp, 'g--', linewidth=2, label=f'Exponential(λ={rate_exp:.4f})')

plt.xlabel('电池寿命（循环次数）')
plt.ylabel('概率密度')
plt.title('寿命分布的拟合比较')
plt.legend()
plt.grid(True, alpha=0.3)

# 3.2 经验CDF与理论CDF
plt.subplot(2, 3, 2)
sorted_lifetimes = np.sort(lifetimes)
ecdf = np.arange(1, len(sorted_lifetimes) + 1) / len(sorted_lifetimes)
plt.step(sorted_lifetimes, ecdf, where='post', label='经验CDF')

cdf_weibull = weibull_min.cdf(sorted_lifetimes, k, scale=lam)
plt.plot(sorted_lifetimes, cdf_weibull, 'r-', alpha=0.7, label='威布尔CDF')

cdf_exp = expon.cdf(sorted_lifetimes, scale=1/rate_exp)
plt.plot(sorted_lifetimes, cdf_exp, 'g--', alpha=0.7, label='指数分布CDF')

plt.xlabel('电池寿命（循环次数）')
plt.ylabel('累积概率')
plt.title('CDF比较')
plt.legend()
plt.grid(True, alpha=0.3)

# 4. 随机变量函数变换：充电周期次数
# 假设每次循环平均使用时长t0=24小时
t0 = 24  # 小时
print(f"\n=== 充电周期分析 ===")
print(f"假设每次循环平均使用时长: {t0} 小时")

# 推导N = T / t0 的分布
# 如果 T ~ Weibull(k, λ)，则 N = T/t0 ~ Weibull(k, λ/t0)
lam_N = lam / t0
print(f"充电周期次数 N ~ Weibull(k={k:.3f}, λ={lam_N:.3f})")
print(f"平均充电周期次数: {lam_N * np.math.gamma(1 + 1/k):.1f}")

# 4.1 充电周期分布
plt.subplot(2, 3, 3)
charge_cycles = lifetimes / t0
plt.hist(charge_cycles, bins=15, density=True, alpha=0.6, edgecolor='black')
x_N = np.linspace(0, max(charge_cycles), 1000)
pdf_N = weibull_min.pdf(x_N, k, scale=lam_N)
plt.plot(x_N, pdf_N, 'b-', linewidth=2)
plt.xlabel('充电周期次数 N')
plt.ylabel('概率密度')
plt.title('充电周期次数分布')
plt.grid(True, alpha=0.3)

# 5. 二维变量分析（模拟）
np.random.seed(42)
n_samples = len(lifetimes)

# 模拟首次充电后电压降V（假设与寿命负相关）
# V = a - b*log(T) + ε
a, b = 4.2, 0.3  # 参数
noise = np.random.normal(0, 0.05, n_samples)
voltage_drop = a - b * np.log(lifetimes) + noise

# 5.1 计算协方差和相关系数
cov_matrix = np.cov(lifetimes, voltage_drop)
corr_coef, p_value = pearsonr(lifetimes, voltage_drop)

print(f"\n=== 二维变量分析 ===")
print(f"协方差矩阵:\n{cov_matrix}")
print(f"相关系数 ρ = {corr_coef:.3f}")
print(f"相关性检验p值 = {p_value:.4f}")

# 5.2 散点图
plt.subplot(2, 3, 4)
plt.scatter(lifetimes, voltage_drop, alpha=0.6)
plt.xlabel('使用寿命 T（循环次数）')
plt.ylabel('首次充电后电压降 V（V）')
plt.title(f'T与V的散点图 (ρ={corr_coef:.3f})')
plt.grid(True, alpha=0.3)

# 添加回归线
z = np.polyfit(lifetimes, voltage_drop, 1)
p = np.poly1d(z)
plt.plot(lifetimes, p(lifetimes), "r--", alpha=0.8)

# 5.3 联合分布直方图
plt.subplot(2, 3, 5)
plt.hist2d(lifetimes, voltage_drop, bins=20, cmap='Blues')
plt.xlabel('使用寿命 T')
plt.ylabel('电压降 V')
plt.title('T与V的联合分布')
plt.colorbar(label='频数')

# 6. Q-Q图检验分布假设
plt.subplot(2, 3, 6)
# 威布尔分布的Q-Q图
weibull_quantiles = weibull_min.ppf(ecdf, k, scale=lam)
plt.scatter(weibull_quantiles, sorted_lifetimes, alpha=0.6)
plt.plot([min(weibull_quantiles), max(weibull_quantiles)], 
         [min(weibull_quantiles), max(weibull_quantiles)], 
         'r--', alpha=0.8)
plt.xlabel('威布尔分布理论分位数')
plt.ylabel('样本分位数')
plt.title('威布尔分布Q-Q图')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('寿命分布建模.png', dpi=300, bbox_inches='tight')
plt.show()

# 7. 保存结果
results_part3 = {
    'weibull_params': weibull_params,
    'correlation': corr_coef,
    'covariance_matrix': cov_matrix
}